This notebook is a simple demo to introduce some of the fundamental design patterns from the OMOP_Alchemy library 

In [1]:
import sqlalchemy as sa
from sqlalchemy.orm import Session, sessionmaker
from omop_alchemy import Base
from omop_alchemy.model.vocabulary import Concept, ConceptView
from omop_alchemy.model.vocabulary import Domain, Vocabulary, Concept_Class
from omop_alchemy import configure_logging, get_engine_name, load_environment, TEST_PATH, ROOT_PATH
from omop_alchemy.cdm.base import bootstrap

In [2]:
# this demo assumes that you have created a .env file in the ROOT_PATH with your database connection string - see .example_dotenv for details

configure_logging()
load_environment()
engine_string = get_engine_name()

engine = sa.create_engine(engine_string, future=True, echo=False)
bootstrap(engine, create=True)

2026-01-03 18:38:55,212 | INFO     | omop_alchemy.omop_alchemy.config | Environment variables loaded from .env file
2026-01-03 18:38:55,213 | INFO     | omop_alchemy.omop_alchemy.config | Database engine configured
2026-01-03 18:38:55,226 | INFO     | omop_alchemy.omop_alchemy.cdm.base.declarative | Bootstrapping OMOP schema (create=True)
2026-01-03 18:38:55,227 | INFO     | omop_alchemy.omop_alchemy.cdm.base.declarative | Schema creation enabled


In [3]:
Session = sessionmaker(bind=engine, future=True)
session = Session()

In [4]:
c = session.query(Concept).first()
c

In [5]:
c.to_dict()

{'concept_id': 1,
 'concept_name': 'Domain',
 'domain_id': 'Metadata',
 'vocabulary_id': 'Domain',
 'concept_class_id': 'Domain',
 'concept_code': 'OMOP generated',
 'valid_start_date': datetime.date(1970, 1, 1),
 'valid_end_date': datetime.date(2099, 12, 31)}

In [6]:
c.to_json()

'{"concept_class_id": "Domain", "concept_code": "OMOP generated", "concept_id": 1, "concept_name": "Domain", "domain_id": "Metadata", "valid_end_date": "2099-12-31", "valid_start_date": "1970-01-01", "vocabulary_id": "Domain"}'

In [7]:
standard_conditions = (
    session.query(Concept)
    .filter(
        Concept.domain_id == "Condition",
        Concept.standard_concept == "S",
    )
    .limit(5)
    .all()
)

[(c.concept_id, c.concept_name, c.standard_concept) for c in standard_conditions]


[(604729, 'Agenesis of calcaneus', 'S'),
 (619070, 'Iris roseola caused by Treponema pallidum', 'S'),
 (751724, 'Ganglioneuroma of nervous system, NOS', 'S'),
 (1076329, 'Neurological complication following vaccination', 'S'),
 (1553072, 'Ewing sarcoma of unknown primary site', 'S')]

`Concept` is the basic class that you should be using for most ETL steps, but for introspection of relationships (including the triggering of lazy loads), `ConceptView` offers much richer expressions.

This is separated to ensure speed of base class is maintained, while optimising the potential benefits of fully-described object relationships

In [8]:
cv = session.query(ConceptView).first()
cv

`domain_id` is the actual string content of the column that was returned from the query already performed, where `cv.domain` returns a related Domain object

In [9]:
cv.domain_id, type(cv.domain_id), cv.domain, type(cv.domain), cv.vocabulary, type(cv.vocabulary)

('Metadata',
 str,
 <Domain Metadata - Metadata>,
 omop_alchemy.model.vocabulary.domain.Domain,
 <Vocabulary Domain>,
 omop_alchemy.model.vocabulary.vocabulary.Vocabulary)

In [10]:
concepts = (
    session.query(ConceptView)
    .filter(ConceptView.vocabulary_id == 'SNOMED')
    .filter(ConceptView.standard_concept == 'S')
    .limit(10)
)

concepts[0].concept_name

'Negative'

In [11]:
# get details about concept dynamically - ancestors, descendants, relationships
for concept in concepts:
    print(concept.concept_id, concept.concept_name, len(concept.ancestors), len(concept.descendants), len(concept.incoming_relationships), len(concept.outgoing_relationships))

9189 Negative 1 1 4 4
9191 Positive 1 1 4 4
507263 Primary lower subciliary and transconjunctival blepharoplasty with skin, muscle and fat excision and canthal sling 2 1 4 4
600817 Sublabial 2 1 4 4
604729 Agenesis of calcaneus 1 1 4 4
607590 Body height 1 1 4 4
619070 Iris roseola caused by Treponema pallidum 1 1 4 4
762840 Arteriovenous graft 2 1 4 4
1073110 Structure of cartilaginous portion of left pharyngotympanic tube 2 1 4 4
1074685 Bone plate submitted as specimen 1 1 4 4


In [12]:
from omop_alchemy.model.clinical import Condition_Occurrence, Condition_OccurrenceView

row = (
    session.query(Condition_Occurrence, Concept)
    .join(Concept, Condition_Occurrence.condition_concept_id == Concept.concept_id)
    .first()
)

row[0].condition_concept_id, row[1].concept_name


(44501475, 'Squamous cell carcinoma, NOS, of branchial cleft')

we don't always want to be using these kinds of implicit joins, which is why they are separated out into View classes, but they can be very useful for exploration, as well as for serialisation to downstream apis

In [ ]:
row = (
    session.query(Condition_OccurrenceView)
    .first()
)

row.condition_concept_id, row.condition_concept.concept_name

(44501475, 'Squamous cell carcinoma, NOS, of branchial cleft')